In [44]:
import pandas as pd
import numpy as np
import time

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn import metrics
from math import sqrt
from catboost import CatBoostRegressor

In [45]:
%%time
def data_loader():
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    sub = pd.read_csv('SampleSubmission.csv')
    return train,test, sub,

Wall time: 0 ns


In [46]:
policy = pd.read_csv("policies.csv")

In [47]:
%%time
train,test, sub,  = data_loader()

Wall time: 84.6 ms


## Feature Engineering

In [49]:
df = pd.concat([train, test]).reset_index(drop=True)
ntrain = train.shape[0]

In [50]:
ntrain = train.shape[0]

In [51]:
df['risk_business'] = df['class_of_business'] + '_' + df['risk_type']
df['product_agent'] = df['agent'] + '_' + df['product']
df['renwal_business'] = df['renewal_frequency'] + '_' + df['class_of_business']
df['product_class'] = df['class_of_business'] + '_' + df['product']

df['year'] = [2022 - int(i.split('-')[0])  for i in df.loss_date]
df['cause'] = df['secondary_cause'] + '_' + df['primary_cause']
df['product_class'] = df['class_of_business'] + '_' + df['product']


df['claim_policy'] = df['claim_number'] + '_' + df['policy_number']
df['business_agent'] = df['class_of_business'] + '_' + df['agent']
df['primary_risk'] = df['primary_cause'] + '_' + df['risk_type']
df['secondary_risk'] = df['secondary_cause'] + '_' + df['risk_type']
df['branch_product'] = pd.Series(df.branch + '_' + df['product'])

id_claim = [i.split('m')[-1] for i in df['claim_number']]
df['id_claim'] = id_claim
df['idclaim_freq'] = df['id_claim'].map(df['id_claim'].value_counts())
df = df.drop('id_claim', axis = 1)

df['loss_date'] = pd.to_datetime(df['loss_date'], format='%Y-%m-%d')

#Engineering Time Features
for attr in ['day', 'month', 'week', 'dayofweek', 'weekofyear', 'days_in_month', 'is_month_start', 'is_month_end', 'dayofyear']:
    df[attr] = getattr(df['loss_date'].dt, attr)
df['is_weekend'] = (df['dayofweek'] >= 5)*1
df['fortnight'] = df['day']%15
df['which_fortnight'] = df['day']//15

C:\Users\Hp\AppData\Local\Temp/ipykernel_19900/1623997607.py:26: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df[attr] = getattr(df['loss_date'].dt, attr)


In [52]:
# Feature Selection after training the model

df = df[['sum_insured', 'product', 'class_of_business', 'risk_type', 'client_type', 
    'renewal_frequency', 'primary_cause', 'secondary_cause', 'branch','renwal_business', 'product_class', 
         'cause', 'year', 'idclaim_freq',
    'day', 'month', 'week', 'dayofweek', 'weekofyear', 'days_in_month', 
    'fortnight', 'which_fortnight', 'branch_product','secondary_risk', 'primary_risk', 'claim_policy', 'business_agent']]

In [53]:
from sklearn.preprocessing import LabelEncoder

In [54]:
features = [feat for feat in list(df) if feat != 'target' ]
cat_feat = np.where(df[features].dtypes != float)[0]

In [55]:
to_encode = [val for index, val in enumerate(features) if index in cat_feat]
df[to_encode] = df[to_encode].apply(LabelEncoder().fit_transform)

In [56]:
new_test = df[train.shape[0]:].reset_index(drop = True)
new_train = df[:train.shape[0]]

In [57]:
lgbm_model = CatBoostRegressor()

fold = model_selection.KFold(n_splits=10)

def metric(y_val, pred):
    return metrics.mean_absolute_error(y_val, pred)

In [70]:
%%time
def modelling_tree(algorithm,yy,X, test_df):
    cv_score=[]
    test_pred=[]
    train_score=[]
    xx = []
    for train_index, test_index in fold.split(X,pd.Series(yy)):
        
        X_trainx, X_val = X.iloc[train_index], X.iloc[test_index]
        y_trainx, y_val = pd.Series(yy).iloc[train_index], pd.Series(yy).iloc[test_index]

        algorithm.fit(X_trainx,y_trainx, eval_set=[(X_trainx,y_trainx),(X_val, y_val)])
        
        train_preds=algorithm.predict(X_trainx)

        train_score.append(metric(y_trainx, np.expm1(train_preds)))
        
        val_preds=algorithm.predict(X_val)
        cv_score.append(metric(y_val, np.expm1(val_preds)))

        test_p = algorithm.predict(test_df)
        test_pred.append(np.expm1(test_p))


    print(f'Training RMSE Score {np.mean(train_score)}')
    print(f'Validation RMSE Score {np.mean(cv_score)}')
    
    return test_pred

Wall time: 0 ns


In [71]:
%%time
test_pred = modelling_tree(algorithm = lgbm_model, yy = np.log1p(train.target), X = new_train, test_df = new_test)

Learning rate set to 0.071173
0:	learn: 3.4642842	test: 3.4642842	test1: 3.5712209	best: 3.5712209 (0)	total: 17.9ms	remaining: 17.9s
1:	learn: 3.4226515	test: 3.4226515	test1: 3.5394811	best: 3.5394811 (1)	total: 23.4ms	remaining: 11.7s
2:	learn: 3.3873472	test: 3.3873472	test1: 3.5110168	best: 3.5110168 (2)	total: 29.1ms	remaining: 9.67s
3:	learn: 3.3575209	test: 3.3575209	test1: 3.4877586	best: 3.4877586 (3)	total: 34.5ms	remaining: 8.6s
4:	learn: 3.3277316	test: 3.3277316	test1: 3.4639858	best: 3.4639858 (4)	total: 40.1ms	remaining: 7.98s
5:	learn: 3.3058165	test: 3.3058165	test1: 3.4475700	best: 3.4475700 (5)	total: 45.7ms	remaining: 7.57s
6:	learn: 3.2865766	test: 3.2865766	test1: 3.4329749	best: 3.4329749 (6)	total: 50.9ms	remaining: 7.23s
7:	learn: 3.2631639	test: 3.2631639	test1: 3.4147141	best: 3.4147141 (7)	total: 56.7ms	remaining: 7.03s
8:	learn: 3.2449199	test: 3.2449199	test1: 3.4026578	best: 3.4026578 (8)	total: 62.1ms	remaining: 6.84s
9:	learn: 3.2293627	test: 3.2293627

90:	learn: 2.8957296	test: 2.8957296	test1: 3.2165920	best: 3.2165920 (90)	total: 538ms	remaining: 5.37s
91:	learn: 2.8938468	test: 2.8938468	test1: 3.2164754	best: 3.2164754 (91)	total: 544ms	remaining: 5.37s
92:	learn: 2.8921741	test: 2.8921741	test1: 3.2168002	best: 3.2164754 (91)	total: 551ms	remaining: 5.37s
93:	learn: 2.8908459	test: 2.8908459	test1: 3.2161454	best: 3.2161454 (93)	total: 557ms	remaining: 5.37s
94:	learn: 2.8893310	test: 2.8893310	test1: 3.2166270	best: 3.2161454 (93)	total: 564ms	remaining: 5.37s
95:	learn: 2.8877965	test: 2.8877965	test1: 3.2165879	best: 3.2161454 (93)	total: 569ms	remaining: 5.36s
96:	learn: 2.8865118	test: 2.8865118	test1: 3.2153906	best: 3.2153906 (96)	total: 574ms	remaining: 5.35s
97:	learn: 2.8850715	test: 2.8850715	test1: 3.2157463	best: 3.2153906 (96)	total: 580ms	remaining: 5.34s
98:	learn: 2.8835096	test: 2.8835096	test1: 3.2169150	best: 3.2153906 (96)	total: 586ms	remaining: 5.33s
99:	learn: 2.8828097	test: 2.8828097	test1: 3.2171574	b

176:	learn: 2.7618058	test: 2.7618058	test1: 3.2028856	best: 3.2002006 (163)	total: 1.06s	remaining: 4.94s
177:	learn: 2.7609576	test: 2.7609576	test1: 3.2032421	best: 3.2002006 (163)	total: 1.07s	remaining: 4.93s
178:	learn: 2.7593430	test: 2.7593430	test1: 3.2034406	best: 3.2002006 (163)	total: 1.07s	remaining: 4.93s
179:	learn: 2.7582670	test: 2.7582670	test1: 3.2035892	best: 3.2002006 (163)	total: 1.08s	remaining: 4.93s
180:	learn: 2.7569089	test: 2.7569089	test1: 3.2033874	best: 3.2002006 (163)	total: 1.09s	remaining: 4.92s
181:	learn: 2.7559019	test: 2.7559019	test1: 3.2036986	best: 3.2002006 (163)	total: 1.09s	remaining: 4.91s
182:	learn: 2.7549632	test: 2.7549632	test1: 3.2030832	best: 3.2002006 (163)	total: 1.1s	remaining: 4.9s
183:	learn: 2.7537141	test: 2.7537141	test1: 3.2030921	best: 3.2002006 (163)	total: 1.1s	remaining: 4.89s
184:	learn: 2.7533473	test: 2.7533473	test1: 3.2031774	best: 3.2002006 (163)	total: 1.11s	remaining: 4.88s
185:	learn: 2.7517090	test: 2.7517090	te

255:	learn: 2.6717701	test: 2.6717701	test1: 3.1992784	best: 3.1992784 (255)	total: 1.61s	remaining: 4.69s
256:	learn: 2.6706307	test: 2.6706307	test1: 3.1990874	best: 3.1990874 (256)	total: 1.62s	remaining: 4.7s
257:	learn: 2.6692312	test: 2.6692312	test1: 3.1990273	best: 3.1990273 (257)	total: 1.63s	remaining: 4.7s
258:	learn: 2.6676120	test: 2.6676120	test1: 3.1976636	best: 3.1976636 (258)	total: 1.64s	remaining: 4.71s
259:	learn: 2.6666684	test: 2.6666684	test1: 3.1977306	best: 3.1976636 (258)	total: 1.65s	remaining: 4.7s
260:	learn: 2.6658645	test: 2.6658645	test1: 3.1975063	best: 3.1975063 (260)	total: 1.66s	remaining: 4.7s
261:	learn: 2.6643496	test: 2.6643496	test1: 3.1971643	best: 3.1971643 (261)	total: 1.67s	remaining: 4.7s
262:	learn: 2.6630511	test: 2.6630511	test1: 3.1971787	best: 3.1971643 (261)	total: 1.68s	remaining: 4.7s
263:	learn: 2.6620399	test: 2.6620399	test1: 3.1973997	best: 3.1971643 (261)	total: 1.68s	remaining: 4.7s
264:	learn: 2.6611362	test: 2.6611362	test1:

348:	learn: 2.5779083	test: 2.5779083	test1: 3.1989483	best: 3.1970203 (325)	total: 2.37s	remaining: 4.42s
349:	learn: 2.5770095	test: 2.5770095	test1: 3.1991839	best: 3.1970203 (325)	total: 2.38s	remaining: 4.42s
350:	learn: 2.5756154	test: 2.5756154	test1: 3.1992234	best: 3.1970203 (325)	total: 2.39s	remaining: 4.42s
351:	learn: 2.5751984	test: 2.5751984	test1: 3.1993741	best: 3.1970203 (325)	total: 2.39s	remaining: 4.41s
352:	learn: 2.5744159	test: 2.5744159	test1: 3.1987731	best: 3.1970203 (325)	total: 2.4s	remaining: 4.4s
353:	learn: 2.5740408	test: 2.5740408	test1: 3.1992776	best: 3.1970203 (325)	total: 2.41s	remaining: 4.39s
354:	learn: 2.5722711	test: 2.5722711	test1: 3.1991476	best: 3.1970203 (325)	total: 2.41s	remaining: 4.39s
355:	learn: 2.5719678	test: 2.5719678	test1: 3.1988415	best: 3.1970203 (325)	total: 2.42s	remaining: 4.38s
356:	learn: 2.5709104	test: 2.5709104	test1: 3.1984455	best: 3.1970203 (325)	total: 2.43s	remaining: 4.37s
357:	learn: 2.5698446	test: 2.5698446	t

...

601:	learn: 2.3823611	test: 2.3823611	test1: 3.0210286	best: 3.0210286 (601)	total: 4.78s	remaining: 3.16s
602:	learn: 2.3816522	test: 2.3816522	test1: 3.0210417	best: 3.0210286 (601)	total: 4.79s	remaining: 3.15s
603:	learn: 2.3813433	test: 2.3813433	test1: 3.0210836	best: 3.0210286 (601)	total: 4.8s	remaining: 3.15s
604:	learn: 2.3808665	test: 2.3808665	test1: 3.0204754	best: 3.0204754 (604)	total: 4.8s	remaining: 3.14s
605:	learn: 2.3806429	test: 2.3806429	test1: 3.0207571	best: 3.0204754 (604)	total: 4.81s	remaining: 3.13s
606:	learn: 2.3795500	test: 2.3795500	test1: 3.0200755	best: 3.0200755 (606)	total: 4.82s	remaining: 3.12s
607:	learn: 2.3785775	test: 2.3785775	test1: 3.0203131	best: 3.0200755 (606)	total: 4.82s	remaining: 3.11s
608:	learn: 2.3778865	test: 2.3778865	test1: 3.0203559	best: 3.0200755 (606)	total: 4.83s	remaining: 3.1s
609:	learn: 2.3772397	test: 2.3772397	test1: 3.0212727	best: 3.0200755 (606)	total: 4.84s	remaining: 3.09s
610:	learn: 2.3767660	test: 2.3767660	te

679:	learn: 2.3273803	test: 2.3273803	test1: 3.0226407	best: 3.0200755 (606)	total: 5.31s	remaining: 2.5s
680:	learn: 2.3268726	test: 2.3268726	test1: 3.0223196	best: 3.0200755 (606)	total: 5.32s	remaining: 2.49s
681:	learn: 2.3262871	test: 2.3262871	test1: 3.0222498	best: 3.0200755 (606)	total: 5.33s	remaining: 2.48s
682:	learn: 2.3255479	test: 2.3255479	test1: 3.0226683	best: 3.0200755 (606)	total: 5.33s	remaining: 2.48s
683:	learn: 2.3248232	test: 2.3248232	test1: 3.0227949	best: 3.0200755 (606)	total: 5.34s	remaining: 2.47s
684:	learn: 2.3242224	test: 2.3242224	test1: 3.0221962	best: 3.0200755 (606)	total: 5.35s	remaining: 2.46s
685:	learn: 2.3228692	test: 2.3228692	test1: 3.0229079	best: 3.0200755 (606)	total: 5.35s	remaining: 2.45s
686:	learn: 2.3223766	test: 2.3223766	test1: 3.0228154	best: 3.0200755 (606)	total: 5.36s	remaining: 2.44s
687:	learn: 2.3217998	test: 2.3217998	test1: 3.0229711	best: 3.0200755 (606)	total: 5.37s	remaining: 2.43s
688:	learn: 2.3207769	test: 2.3207769	

764:	learn: 2.2706339	test: 2.2706339	test1: 3.0283123	best: 3.0200755 (606)	total: 5.87s	remaining: 1.8s
765:	learn: 2.2702605	test: 2.2702605	test1: 3.0282175	best: 3.0200755 (606)	total: 5.89s	remaining: 1.8s
766:	learn: 2.2700981	test: 2.2700981	test1: 3.0285260	best: 3.0200755 (606)	total: 5.9s	remaining: 1.79s
767:	learn: 2.2698607	test: 2.2698607	test1: 3.0283895	best: 3.0200755 (606)	total: 5.91s	remaining: 1.78s
768:	learn: 2.2692418	test: 2.2692418	test1: 3.0277162	best: 3.0200755 (606)	total: 5.91s	remaining: 1.78s
769:	learn: 2.2682920	test: 2.2682920	test1: 3.0273532	best: 3.0200755 (606)	total: 5.93s	remaining: 1.77s
770:	learn: 2.2675703	test: 2.2675703	test1: 3.0275240	best: 3.0200755 (606)	total: 5.95s	remaining: 1.77s
771:	learn: 2.2671129	test: 2.2671129	test1: 3.0275265	best: 3.0200755 (606)	total: 5.96s	remaining: 1.76s
772:	learn: 2.2662088	test: 2.2662088	test1: 3.0275607	best: 3.0200755 (606)	total: 5.98s	remaining: 1.76s
773:	learn: 2.2654170	test: 2.2654170	te

862:	learn: 2.2177432	test: 2.2177432	test1: 3.0328232	best: 3.0200755 (606)	total: 6.55s	remaining: 1.04s
863:	learn: 2.2170844	test: 2.2170844	test1: 3.0329399	best: 3.0200755 (606)	total: 6.56s	remaining: 1.03s
864:	learn: 2.2159958	test: 2.2159958	test1: 3.0336610	best: 3.0200755 (606)	total: 6.56s	remaining: 1.02s
865:	learn: 2.2152860	test: 2.2152860	test1: 3.0334495	best: 3.0200755 (606)	total: 6.57s	remaining: 1.02s
866:	learn: 2.2149460	test: 2.2149460	test1: 3.0334632	best: 3.0200755 (606)	total: 6.58s	remaining: 1.01s
867:	learn: 2.2142601	test: 2.2142601	test1: 3.0333676	best: 3.0200755 (606)	total: 6.58s	remaining: 1s
868:	learn: 2.2138561	test: 2.2138561	test1: 3.0335048	best: 3.0200755 (606)	total: 6.59s	remaining: 993ms
869:	learn: 2.2129345	test: 2.2129345	test1: 3.0335596	best: 3.0200755 (606)	total: 6.59s	remaining: 985ms
870:	learn: 2.2126159	test: 2.2126159	test1: 3.0340435	best: 3.0200755 (606)	total: 6.6s	remaining: 977ms
871:	learn: 2.2118648	test: 2.2118648	tes

954:	learn: 2.1651987	test: 2.1651987	test1: 3.0344925	best: 3.0200755 (606)	total: 7.09s	remaining: 334ms
955:	learn: 2.1645859	test: 2.1645859	test1: 3.0345550	best: 3.0200755 (606)	total: 7.09s	remaining: 326ms
956:	learn: 2.1641059	test: 2.1641059	test1: 3.0342917	best: 3.0200755 (606)	total: 7.1s	remaining: 319ms
957:	learn: 2.1635299	test: 2.1635299	test1: 3.0341429	best: 3.0200755 (606)	total: 7.11s	remaining: 312ms
958:	learn: 2.1633891	test: 2.1633891	test1: 3.0342165	best: 3.0200755 (606)	total: 7.11s	remaining: 304ms
959:	learn: 2.1622385	test: 2.1622385	test1: 3.0348055	best: 3.0200755 (606)	total: 7.12s	remaining: 297ms
960:	learn: 2.1616490	test: 2.1616490	test1: 3.0350344	best: 3.0200755 (606)	total: 7.12s	remaining: 289ms
961:	learn: 2.1610784	test: 2.1610784	test1: 3.0355531	best: 3.0200755 (606)	total: 7.13s	remaining: 282ms
962:	learn: 2.1606469	test: 2.1606469	test1: 3.0357966	best: 3.0200755 (606)	total: 7.14s	remaining: 274ms
963:	learn: 2.1603774	test: 2.1603774	

In [72]:
cat_sub = np.mean(np.exp(test_pred) +1, axis = 0)
sub['target'] = cat_sub
sub['target'].describe()
sub.to_csv('Final_solution.csv', index = False)

C:\Users\Hp\AppData\Local\Temp/ipykernel_19900/539007523.py:1: RuntimeWarning: overflow encountered in exp
  cat_sub = np.mean(np.exp(test_pred) +1, axis = 0)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3961: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
